In [315]:
import json
import graphviz
import copy

In [316]:
with open('../testcases/test23/NFA.json') as f:
    nfa = json.load(f)


terminating_states = set()
#loop through the states, and make each transition a list if it isn't already
for state in nfa:
    if state == 'startingState':
        nfa[state] = [nfa[state]]
        continue
    
    for transition in nfa[state]:
        if transition == 'isTerminatingState':
            if nfa[state][transition] == True:
                terminating_states.add(state)
            continue
        if type(nfa[state][transition]) != list:
            nfa[state][transition] = [nfa[state][transition]]

print(nfa)
print(terminating_states)
starting_state_origin = nfa['startingState']

{'startingState': ['S1'], 'S1': {'isTerminatingState': False, 'e': ['S2']}, 'S2': {'isTerminatingState': False, 'epsilon': ['S3']}, 'S3': {'isTerminatingState': False, 'm': ['S4']}, 'S4': {'isTerminatingState': False, 'epsilon': ['S5']}, 'S5': {'isTerminatingState': False, 'p': ['S6']}, 'S6': {'isTerminatingState': False, 'epsilon': ['S7']}, 'S7': {'isTerminatingState': False, 'l': ['S8']}, 'S8': {'isTerminatingState': False, 'epsilon': ['S9']}, 'S9': {'isTerminatingState': False, 'o': ['S10']}, 'S10': {'isTerminatingState': False, 'epsilon': ['S11']}, 'S11': {'isTerminatingState': False, 'y': ['S12']}, 'S12': {'isTerminatingState': False, 'epsilon': ['S49']}, 'S49': {'isTerminatingState': False, 'epsilon': ['S41', 'S39']}, 'S50': {'isTerminatingState': True}, 'S41': {'isTerminatingState': False, 'a': ['S42']}, 'S42': {'isTerminatingState': False, 'epsilon': ['S43']}, 'S43': {'isTerminatingState': False, 'b': ['S44']}, 'S44': {'isTerminatingState': False, 'epsilon': ['S45']}, 'S45': {'

## NFA 2 DFA

In [317]:
#global variable to store the closures of each state
#used to avoid recalculating the closure of a state
#DP approach
global_closuers = dict()

def get_possible_inputs(states):
    possible_transitions = set()
    # print(states)
    for state in states:
        for symbol in nfa[state]:
            if symbol != "epsilon" and symbol != "isTerminatingState":
                # print(symbol)
                possible_transitions.add(symbol)
    return possible_transitions

def epsilon_closure(nfa, state, prev_closures):
    # print(state)
    if state in global_closuers:
        return global_closuers[state]

    #initialize the closure of the current state
    # with the current state
    closure = set()
    closure.add(state)
    prev_closures.add(state)
    # add epsilon transitions of the current state
    epsilon_transitions = set(nfa[state].get("epsilon", []))
    
    # print(epsilon_transitions)
    for s in epsilon_transitions:
        #union of the current closure and the closure of the next state
        if s not in prev_closures:
            closure |= epsilon_closure(nfa, s, prev_closures)

    global_closuers[state] = closure
    return closure

def move(nfa, states, symbol):
    result = set()
    for state in states:
        if symbol in nfa[state]:
            if symbol == "epsilon" or symbol == "isTerminatingState":
                continue
            #print(nfa[state][symbol])
            result |= set(nfa[state][symbol])
    return result

def nfa_to_dfa(nfa):
    dfa = {}
    #used to use as a key in dfa, sets are not hashable
    #because they are mutable, but frozensets are immutable
    starting_state = nfa["startingState"][0]
    start_state = frozenset(epsilon_closure(nfa, starting_state, set()))

    dfa["startingState"] = start_state
    dfa[start_state] = {}
    queue = [start_state]
    while queue:
        is_terminating = False
        current_state = queue.pop(0)
        if current_state & terminating_states:
            is_terminating = True
        dfa[current_state]["isTerminatingState"] = is_terminating
            
        possible_inputs = get_possible_inputs(current_state)
        for minput in possible_inputs:
            possible_transitions = move(nfa, current_state, minput)
            next_states_closures = set()
            for next_state in possible_transitions:
                next_states_closures |= epsilon_closure(nfa, next_state, set())
                
            closure = frozenset(next_states_closures)
            if closure not in dfa:
                dfa[closure] = {}
                queue.append(closure)
            dfa[current_state][minput] = closure
            
    return dfa

dfa = nfa_to_dfa(nfa)
print(dfa)

{'startingState': frozenset({'S1'}), frozenset({'S1'}): {'isTerminatingState': False, 'e': frozenset({'S2', 'S3'})}, frozenset({'S2', 'S3'}): {'isTerminatingState': False, 'm': frozenset({'S5', 'S4'})}, frozenset({'S5', 'S4'}): {'isTerminatingState': False, 'p': frozenset({'S7', 'S6'})}, frozenset({'S7', 'S6'}): {'isTerminatingState': False, 'l': frozenset({'S9', 'S8'})}, frozenset({'S9', 'S8'}): {'isTerminatingState': False, 'o': frozenset({'S11', 'S10'})}, frozenset({'S11', 'S10'}): {'isTerminatingState': False, 'y': frozenset({'S21', 'S31', 'S41', 'S33', 'S49', 'S17', 'S39', 'S13', 'S23', 'S12'})}, frozenset({'S21', 'S31', 'S41', 'S33', 'S49', 'S17', 'S39', 'S13', 'S23', 'S12'}): {'isTerminatingState': False, 'm': frozenset({'S25', 'S24'}), 'i': frozenset({'S34', 'S35'}), 'e': frozenset({'S14', 'S19', 'S18', 'S15'}), 'a': frozenset({'S42', 'S43'})}, frozenset({'S25', 'S24'}): {'isTerminatingState': False, 'e': frozenset({'S26', 'S27'})}, frozenset({'S34', 'S35'}): {'isTerminatingSta

## Clean States

In [318]:
def clean_DFA(dfa):
    index = 1
    dfa_cleaned = {}
    mapping = {}

    mapping[dfa["startingState"]] = "S0"

    for state in dfa:
        
        if state == "startingState":
            continue

        if mapping.get(state) == None:      
            state_name = "S" + str(index)
            mapping[state] = state_name
            index += 1


    for state in dfa:
        if state == "startingState":
            dfa_cleaned[state] = mapping[dfa[state]]
            continue
        
        dfa_cleaned[mapping[state]] = {"isTerminatingState": dfa[state]["isTerminatingState"]}

        for transition in dfa[state]:
            if transition == "isTerminatingState":
                continue

            dfa_cleaned[mapping[state]][transition] = {mapping[dfa[state][transition]]}

    return dfa_cleaned

dfa_cleaned = clean_DFA(dfa)
starting_state_origin = dfa_cleaned["startingState"]
print(dfa_cleaned)

{'startingState': 'S0', 'S0': {'isTerminatingState': False, 'e': {'S1'}}, 'S1': {'isTerminatingState': False, 'm': {'S2'}}, 'S2': {'isTerminatingState': False, 'p': {'S3'}}, 'S3': {'isTerminatingState': False, 'l': {'S4'}}, 'S4': {'isTerminatingState': False, 'o': {'S5'}}, 'S5': {'isTerminatingState': False, 'y': {'S6'}}, 'S6': {'isTerminatingState': False, 'm': {'S7'}, 'i': {'S8'}, 'e': {'S9'}, 'a': {'S10'}}, 'S7': {'isTerminatingState': False, 'e': {'S11'}}, 'S8': {'isTerminatingState': False, 'n': {'S12'}}, 'S9': {'isTerminatingState': False, 'r': {'S13'}, 'e': {'S14'}}, 'S10': {'isTerminatingState': False, 'b': {'S15'}}, 'S11': {'isTerminatingState': False, 'n': {'S16'}}, 'S12': {'isTerminatingState': False, 'g': {'S17'}}, 'S13': {'isTerminatingState': True}, 'S14': {'isTerminatingState': True}, 'S15': {'isTerminatingState': False, 'l': {'S18'}}, 'S16': {'isTerminatingState': False, 't': {'S19'}}, 'S17': {'isTerminatingState': True}, 'S18': {'isTerminatingState': False, 'e': {'S20'

## Draw DFA and Write DFA

In [319]:
def draw_dfa(dfa, name='dfa_graph'):
    dot = graphviz.Digraph()
    starting_state = dfa['startingState']
    for state, transitions in dfa.items():
        if state == 'startingState':
            continue 
        if state == starting_state:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle', color='blue')
        else:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle')

        for symbol, next_states in transitions.items():
            if symbol != 'isTerminatingState' and symbol != 'startingState':
                for next_state in next_states:
                    dot.edge(state, next_state, label=symbol)

    dot.render(name, format='png', cleanup=True)

def write_dfa(dfa_min, name='minimized_DFA.json'):
    #convert transitions to list to be serializable
    for state in dfa_min:
        if state == 'startingState':
            dfa_min[state] = [dfa_min[state]]
            continue

        for transition in dfa_min[state]:
            if transition == 'isTerminatingState':
                continue
            dfa_min[state][transition] = list(dfa_min[state][transition])
            
    with open(name, 'w') as _:
                json.dump(dfa_min, _, indent=4)

In [320]:
draw_dfa(dfa_cleaned, 'dfa_graph1')

## Minimize DFA

In [321]:
#at the start, partition of 1 is the set of all terminating states
#and partition 2 is the set of all non-terminating states
def create_partitions(dfa):
    partitions = {1: set(), 2: set()}
    for state in dfa:
        if state == "startingState":
            continue
        if dfa[state]["isTerminatingState"]:
            partitions[1].add(state)
        else:
            partitions[2].add(state)

    #if one of them is empty, just remove it
    if not partitions[1]:
        partitions.pop(1)

    if not partitions[2]:
        partitions.pop(2)
        
    return partitions

def minimize_dfa(dfa, partitions):
    new_partitions = copy.deepcopy(partitions)
    while True:
        try:
            for partition in partitions:
                
                list_splited_partitions = []

                if len(partitions[partition]) == 1:
                    continue

                pivot_state = next(iter(partitions[partition]))
            
                for state in partitions[partition]:
                    if pivot_state == state:
                        continue

                    if not are_equivalent(dfa, pivot_state, state, new_partitions[partition]):
                        new_partitions[partition].remove(state)

                        for splited_partition in list_splited_partitions:
                            temp_state = next(iter(new_partitions[splited_partition]))
                            if are_equivalent(dfa, state, temp_state, new_partitions[splited_partition]):
                                new_partitions[splited_partition].add(state)
                                break
                        else:   
                            list_splited_partitions.append(len(new_partitions)+1)
                            new_partitions[len(new_partitions) + 1] = {state}
                                     
            if new_partitions == partitions:
                raise 

            partitions = copy.deepcopy(new_partitions)
        except Exception as e:
            break

    return partitions

def are_equivalent(dfa, state1, state2, partition):
    #check if the length of the transitions of the two states are the same
    if len(dfa[state1]) != len(dfa[state2]):
        return False

    for symbol in dfa[state1]:
        if symbol == "isTerminatingState":
            continue

        if dfa[state2].get(symbol) == None:
            return False

        if (dfa[state1][symbol] != dfa[state2][symbol]) \
            and not (next(iter(dfa[state1][symbol])) in partition and next(iter(dfa[state2][ symbol])) in partition):
            return False
         
    return True                

def merge_partitions(dfa, partitions):
    # merged_dfa = {"startingState": partitions[1].pop()}
    merged_dfa = {}
    pivots = {}
    dfa = copy.deepcopy(dfa)
    
    for key, partition in partitions.items():
        pivot_state = next(iter(partition))
        merged_transitions = {"isTerminatingState": dfa[pivot_state]["isTerminatingState"]}
        pivots[key] = pivot_state

        for state in partition:
            if state == starting_state_origin:
                merged_dfa["startingState"] = pivot_state

            merged_transitions |= dfa[state]

        merged_dfa[pivot_state] = merged_transitions

    #loop over all transitions, and if the transition contains any state
    #that belongs to a partition, change it to the pivot state
    for state in merged_dfa:
        if state == "startingState":
            if not merged_dfa[state] in pivots.values():
                pivot_pratition = find_partition(partitions, merged_dfa[state])
                merged_dfa[state] = pivots[pivot_pratition]
            continue

        for symbol in merged_dfa[state]:
            if symbol == "isTerminatingState":
                continue

            if not next(iter(merged_dfa[state][symbol])) in pivots.values():
                pivot_partition = find_partition(partitions, next(iter(merged_dfa[state][symbol])))
                merged_dfa[state][symbol].discard(next(iter(merged_dfa[state][symbol])))
                merged_dfa[state][symbol].add(pivots[pivot_partition])
           
    return merged_dfa

                
def find_partition(partitions, state):
    for partition in partitions:
        if state in partitions[partition]:
            return partition

    return None

def minimize(dfa):
    partitions = create_partitions(dfa)
    minimized_dfa = minimize_dfa(dfa, partitions)
    merged_dfa = merge_partitions(dfa, minimized_dfa)
    return merged_dfa

dfa_min = minimize(dfa_cleaned)
write_dfa(dfa_min)
draw_dfa(dfa_min, 'dfa_graph_min')